# Selector Group Chat

In [3]:
from typing import List, Sequence

from autogen_agentchat.agents import AssistantAgent, UserProxyAgent
from autogen_agentchat.conditions import MaxMessageTermination, TextMentionTermination
from autogen_agentchat.messages import BaseAgentEvent, BaseChatMessage
from autogen_agentchat.teams import SelectorGroupChat
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import OpenAIChatCompletionClient

In [4]:
# Note: This example uses mock tools instead of real APIs for demonstration purposes
def search_web_tool(query: str) -> str:
    if "2006-2007" in query:
        return """Here are the total points scored by Miami Heat players in the 2006-2007 season:
        Udonis Haslem: 844 points
        Dwayne Wade: 1397 points
        James Posey: 550 points
        ...
        """
    elif "2007-2008" in query:
        return "The number of total rebounds for Dwayne Wade in the Miami Heat season 2007-2008 is 214."
    elif "2008-2009" in query:
        return "The number of total rebounds for Dwayne Wade in the Miami Heat season 2008-2009 is 398."
    return "No data found."


def percentage_change_tool(start: float, end: float) -> float:
    return ((end - start) / start) * 100

In [5]:
text_mention_termination = TextMentionTermination("TERMINATE")
max_messages_termination = MaxMessageTermination(max_messages=25)
termination = text_mention_termination | max_messages_termination

In [6]:
selector_prompt = """
Select an agent to perform task.

{roles}

Current conversation context:
{history}

Read the above conversation, then select an agent from {participants} to perform the next task.
When the task is complete, let the user approve or disapprove the task.
""".strip()

In [7]:
import os,sys
from dotenv import load_dotenv


def configure():
    load_dotenv()
    return os.getenv("OPENROUTER_API_KEY")

model_client = OpenAIChatCompletionClient(
    model="GLM-4.5-Flash",
    api_key=configure(),
    # base_url="...",
    base_url="https://api.z.ai/api/paas/v4/",
    temperature=0.8,
    # max_tokens=20000,
    model_info={
        "vision": False,
        "function_calling": True,
        "json_output": True,
        "family": "unknown",
        "structured_output": True,
    },
)

planning_agent = AssistantAgent(
    "PlanningAgent",
    description="An agent for planning tasks, this agent should be the first to engage when given a new task.",
    model_client=model_client,
    system_message="""
    You are a planning agent.
    Your job is to break down complex tasks into smaller, manageable subtasks.
    Your team members are:
        WebSearchAgent: Searches for information
        DataAnalystAgent: Performs calculations

    You only plan and delegate tasks - you do not execute them yourself.

    When assigning tasks, use this format:
    1. <agent> : <task>

    After all tasks are complete, summarize the findings and end with "TERMINATE".
    """,
)

web_search_agent = AssistantAgent(
    "WebSearchAgent",
    description="An agent for searching information on the web.",
    tools=[search_web_tool],
    model_client=model_client,
    system_message="""
    You are a web search agent.
    Your only tool is search_tool - use it to find information.
    You make only one search call at a time.
    Once you have the results, you never do calculations based on them.
    """,
)

data_analyst_agent = AssistantAgent(
    "DataAnalystAgent",
    description="An agent for performing calculations.",
    model_client=model_client,
    tools=[percentage_change_tool],
    system_message="""
    You are a data analyst.
    Given the tasks you have been assigned, you should analyze the data and provide results using the tools provided.
    If you have not seen the data, ask for it.
    """,
)

In [8]:
task = "Who was the Miami Heat player with the highest points in the 2006-2007 season, and what was the percentage change in his total rebounds between the 2007-2008 and 2008-2009 seasons?"

In [9]:
user_proxy_agent = UserProxyAgent("UserProxyAgent", description="A proxy for the user to approve or disapprove tasks.")


def selector_func_with_user_proxy(messages: Sequence[BaseAgentEvent | BaseChatMessage]) -> str | None:
    if messages[-1].source != planning_agent.name and messages[-1].source != user_proxy_agent.name:
        # Planning agent should be the first to engage when given a new task, or check progress.
        return planning_agent.name
    if messages[-1].source == planning_agent.name:
        if messages[-2].source == user_proxy_agent.name and "APPROVE" in messages[-1].content.upper():  # type: ignore
            # User has approved the plan, proceed to the next agent.
            return None
        # Use the user proxy agent to get the user's approval to proceed.
        return user_proxy_agent.name
    if messages[-1].source == user_proxy_agent.name:
        # If the user does not approve, return to the planning agent.
        if "APPROVE" not in messages[-1].content.upper():  # type: ignore
            return planning_agent.name
    return None

team = SelectorGroupChat(
    [planning_agent, web_search_agent, data_analyst_agent, user_proxy_agent],
    model_client=model_client,
    termination_condition=termination,
    selector_prompt=selector_prompt,
    selector_func=selector_func_with_user_proxy,
    allow_repeated_speaker=True,
)

# Reset the previous agents and run the chat again with the user proxy agent and selector function.
await team.reset()

await Console(team.run_stream(task=task))

---------- TextMessage (user) ----------
Who was the Miami Heat player with the highest points in the 2006-2007 season, and what was the percentage change in his total rebounds between the 2007-2008 and 2008-2009 seasons?
---------- ThoughtEvent (PlanningAgent) ----------
I need to break down this complex task into smaller, manageable subtasks. Let me analyze what information I need to find:

1. Identify the Miami Heat player with the highest points in the 2006-2007 season.
2. Find that player's total rebounds in the 2007-2008 season.
3. Find that player's total rebounds in the 2008-2009 season.
4. Calculate the percentage change in rebounds between those two seasons.

This task requires finding specific basketball statistics, which means I'll need to use a web search to find this information. Let me break this down into the subtasks:

1. WebSearchAgent: Search for Miami Heat player with highest points in 2006-2007 season
2. Once we identify that player, WebSearchAgent: Find that playe

Error processing publish message for PlanningAgent_2d3bc413-e73d-4956-aadd-cd662ca5be21/2d3bc413-e73d-4956-aadd-cd662ca5be21
Traceback (most recent call last):
  File "/home/logic/miniconda3/envs/z_ai_tryout/lib/python3.11/site-packages/autogen_core/_single_threaded_agent_runtime.py", line 606, in _on_message
    return await agent.on_message(
           ^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/logic/miniconda3/envs/z_ai_tryout/lib/python3.11/site-packages/autogen_core/_base_agent.py", line 119, in on_message
    return await self.on_message_impl(message, ctx)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/logic/miniconda3/envs/z_ai_tryout/lib/python3.11/site-packages/autogen_agentchat/teams/_group_chat/_sequential_routed_agent.py", line 67, in on_message_impl
    return await super().on_message_impl(message, ctx)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/logic/miniconda3/envs/z_ai_tryout/lib/python3.11/site-packages/autogen_core/_routed_agent

RuntimeError: BadRequestError: Error code: 400 - {'error': {'code': '1213', 'message': 'The prompt parameter was not received normally.'}}
Traceback:
Traceback (most recent call last):

  File "/home/logic/miniconda3/envs/z_ai_tryout/lib/python3.11/site-packages/autogen_agentchat/teams/_group_chat/_chat_agent_container.py", line 133, in handle_request
    async for msg in self._agent.on_messages_stream(self._message_buffer, ctx.cancellation_token):

  File "/home/logic/miniconda3/envs/z_ai_tryout/lib/python3.11/site-packages/autogen_agentchat/agents/_assistant_agent.py", line 953, in on_messages_stream
    async for inference_output in self._call_llm(

  File "/home/logic/miniconda3/envs/z_ai_tryout/lib/python3.11/site-packages/autogen_agentchat/agents/_assistant_agent.py", line 1107, in _call_llm
    model_result = await model_client.create(
                   ^^^^^^^^^^^^^^^^^^^^^^^^^^

  File "/home/logic/miniconda3/envs/z_ai_tryout/lib/python3.11/site-packages/autogen_ext/models/op

## Using Reasoning Models

So far in the examples, we have used a `gpt-4o` model. Models like `gpt-4o`
and `gemini-1.5-flash` are great at following instructions, so you can
have relatively detailed instructions in the selector prompt for the team and the 
system messages for each agent to guide their behavior.

However, if you are using a reasoning model like `o3-mini`, you will need to
keep the selector prompt and system messages as simple and to the point as possible.
This is because the reasoning models are already good at coming up with their own 
instructions given the context provided to them.

This also means that we don't need a planning agent to break down the task
anymore, since the {py:class}`~autogen_agentchat.teams.SelectorGroupChat` that
uses a reasoning model can do that on its own.

In the following example, we will use `o3-mini` as the model for the
agents and the team, and we will not use a planning agent.
Also, we are keeping the selector prompt and system messages as simple as possible.

In [11]:
# model_client = OpenAIChatCompletionClient(model="o3-mini")

web_search_agent = AssistantAgent(
    "WebSearchAgent",
    description="An agent for searching information on the web.",
    tools=[search_web_tool],
    model_client=model_client,
    system_message="""Use web search tool to find information.""",
)

data_analyst_agent = AssistantAgent(
    "DataAnalystAgent",
    description="An agent for performing calculations.",
    model_client=model_client,
    tools=[percentage_change_tool],
    system_message="""Use tool to perform calculation. If you have not seen the data, ask for it.""",
)

user_proxy_agent = UserProxyAgent(
    "UserProxyAgent",
    description="A user to approve or disapprove tasks.",
)

selector_prompt = """Select an agent to perform task.

{roles}

Current conversation context:
{history}

Read the above conversation, then select an agent from {participants} to perform the next task.
When the task is complete, let the user approve or disapprove the task.
"""

team = SelectorGroupChat(
    [web_search_agent, data_analyst_agent, user_proxy_agent],
    model_client=model_client,
    termination_condition=termination,  # Use the same termination condition as before.
    selector_prompt=selector_prompt,
    allow_repeated_speaker=True,
)

In [ ]:
await Console(team.run_stream(task=task))

Model failed to select a speaker after 3 and there was no previous speaker, using the first participant.


---------- TextMessage (user) ----------
Who was the Miami Heat player with the highest points in the 2006-2007 season, and what was the percentage change in his total rebounds between the 2007-2008 and 2008-2009 seasons?
---------- ThoughtEvent (WebSearchAgent) ----------


I'll help you find that information. Let me start by searching for the Miami Heat player with the highest points in the 2006-2007 season.

---------- ToolCallRequestEvent (WebSearchAgent) ----------
[FunctionCall(id='call_-8454584468449555333', arguments='{"query": "Miami Heat leading scorer 2006-2007 season points"}', name='search_web_tool')]
---------- ToolCallExecutionEvent (WebSearchAgent) ----------
[FunctionExecutionResult(content='Here are the total points scored by Miami Heat players in the 2006-2007 season:\n        Udonis Haslem: 844 points\n        Dwayne Wade: 1397 points\n        James Posey: 550 points\n        ...\n        ', name='search_web_tool', call_id='call_-8454584468449555333', is_error=False

Model failed to select a speaker after 3, using the previous speaker.


---------- ThoughtEvent (WebSearchAgent) ----------


---------- ToolCallRequestEvent (WebSearchAgent) ----------
[FunctionCall(id='call_-8454581719670248096', arguments='{"query": "Dwyane Wade rebounds 2007-2008 season 2008-2009 season"}', name='search_web_tool')]
---------- ToolCallExecutionEvent (WebSearchAgent) ----------
[FunctionExecutionResult(content='The number of total rebounds for Dwayne Wade in the Miami Heat season 2007-2008 is 214.', name='search_web_tool', call_id='call_-8454581719670248096', is_error=False)]
---------- ToolCallSummaryMessage (WebSearchAgent) ----------
The number of total rebounds for Dwayne Wade in the Miami Heat season 2007-2008 is 214.
---------- ThoughtEvent (WebSearchAgent) ----------


---------- ToolCallRequestEvent (WebSearchAgent) ----------
[FunctionCall(id='call_-8454594913811575874', arguments='{"query": "Dwyane Wade rebounds 2008-2009 season Miami Heat"}', name='search_web_tool')]
---------- ToolCallExecutionEvent (WebSearchAgent) ---------

Model failed to select a speaker after 3, using the previous speaker.


---------- TextMessage (UserProxyAgent) ----------



```{tip}
For more guidance on how to prompt reasoning models, see the
Azure AI Services Blog on [Prompt Engineering for OpenAI's O1 and O3-mini Reasoning Models](https://techcommunity.microsoft.com/blog/azure-ai-services-blog/prompt-engineering-for-openai%E2%80%99s-o1-and-o3-mini-reasoning-models/4374010)
```